# Uploading documents

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from dotenv import load_dotenv
import os

load_dotenv()  # Automatically loads variables from .env file

In [ ]:
# === 1. KONFIGURATION ===
storage_account_url = "https://elternlebenpocteam5us.blob.core.windows.net"
container_name = "elternleben-de-website"
local_folder_path = "../../../n3xtcoder/ai4impact-elternleben"

In [ ]:
# === 2. VERBINDUNG AUFBAUEN ===
credential = DefaultAzureCredential()
blob_service_client = BlobServiceClient(account_url=storage_account_url, credential=credential)


In [ ]:
# === 3. CONTAINER ERSTELLEN (falls nicht vorhanden) ===
try:
    container_client = blob_service_client.create_container(container_name)
    print(f"Container '{container_name}' wurde erstellt.")
except Exception as e:
    print(f"Container '{container_name}' existiert wahrscheinlich schon. {e}")
    container_client = blob_service_client.get_container_client(container_name)

In [ ]:
# === 4. REKURSIVES HOCHLADEN DER MARKDOWN-DATEIEN ===
for root, dirs, files in os.walk(local_folder_path):
    for filename in files:
        if filename.endswith(".md"):
            file_path_on_disk = os.path.join(root, filename)
            
            # Pfad innerhalb des Containers (ohne lokalen Basisordner)
            relative_path = os.path.relpath(file_path_on_disk, start=local_folder_path)
            blob_client = container_client.get_blob_client(blob=relative_path.replace("\\", "/"))  # Wichtig für Windows
            
            # Datei hochladen
            with open(file_path_on_disk, "rb") as data:
                blob_client.upload_blob(data, overwrite=True)
                print(f"Hochgeladen: {relative_path}")